In [1]:
from pydantic import BaseModel, Field
from typing import List
import instructor
from openai import OpenAI
from langchain_core.messages import AIMessage, ToolMessage

import json

from typing import List, Dict, Any, Annotated, Optional
from operator import add
from jinja2 import Template

In [7]:
from dotenv import load_dotenv
load_dotenv()  # This loads variables from .env into the environment

import os
api_key = os.getenv("LANGSMITH_API_KEY")
if not api_key:
    raise ValueError("LANGSMITH_API_KEY environment variable is not set. Please set it before running this cell.")

In [2]:
import os

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Please enter your OpenAI API key: ").strip()
    os.environ["OPENAI_API_KEY"] = api_key
    if not api_key:
        raise ValueError("API key is required.")

### Create Coordinator evaluation dataset

In [2]:
def lc_messages_to_regular_messages(msg):

    if isinstance(msg, dict):
        
        if msg.get("role") == "user":
            return {"role": "user", "content": msg["content"]}
        elif msg.get("role") == "assistant":
            return {"role": "assistant", "content": msg["content"]}
        elif msg.get("role") == "tool":
            return {
                "role": "tool", 
                "content": msg["content"], 
                "tool_call_id": msg.get("tool_call_id")
            }
        
    elif isinstance(msg, AIMessage):

        result = {
            "role": "assistant",
            "content": msg.content
        }
        
        if hasattr(msg, 'tool_calls') and msg.tool_calls and len(msg.tool_calls) > 0 and not msg.tool_calls[0].get("name").startswith("functions."):
            result["tool_calls"] = [
                {
                    "id": tc["id"],
                    "type": "function",
                    "function": {
                        "name": tc["name"].replace("functions.", ""),
                        "arguments": json.dumps(tc["args"])
                    }
                }
                for tc in msg.tool_calls
            ]
            
        return result
    
    elif isinstance(msg, ToolMessage):

        return {"role": "tool", "content": msg.content, "tool_call_id": msg.tool_call_id}
    
    else:

        return {"role": "user", "content": str(msg)}


In [3]:
class MCPToolCall(BaseModel):
    name: str
    arguments: dict
    server: str


class ToolCall(BaseModel):
    name: str
    arguments: dict


class RAGUsedContext(BaseModel):
    id: str
    description: str


class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")


class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""
    product_qa_iteration: int = Field(default=0)
    shopping_cart_iteration: int = Field(default=0)
    coordinator_iteration: int = Field(default=0)
    product_qa_final_answer: bool = Field(default=False)
    shopping_cart_final_answer: bool = Field(default=False)
    coordinator_final_answer: bool = Field(default=False)
    product_qa_available_tools: List[Dict[str, Any]] = []
    shopping_cart_available_tools: List[Dict[str, Any]] = []
    mcp_tool_calls: Optional[List[MCPToolCall]] = Field(default_factory=list)
    tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    retrieved_context_ids: List[RAGUsedContext] = Field(default_factory=list)
    trace_id: str = ""
    user_id: str = ""
    cart_id: str = ""
    plan: List[Delegation] = Field(default_factory=list)
    next_agent: str = ""

In [4]:
def coordinator_agent_node(state) -> dict:

   prompt_template = """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true

Additional instructions:

- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

   prompt = Template(prompt_template).render()

   messages = state.messages

   conversation = []

   for msg in messages:
      conversation.append(lc_messages_to_regular_messages(msg))

   client = instructor.from_openai(OpenAI())

   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0,
   )

   if response.final_answer:
      ai_message = [AIMessage(
         content=response.answer,
      )]
   else:
      ai_message = []

   return {
      "messages": ai_message,
      "answer": response.answer,
      "next_agent": response.next_agent,
      "plan": response.plan,
      "coordinator_final_answer": response.final_answer,
      "coordinator_iteration": state.coordinator_iteration + 1,
      "trace_id": "",
   }


In [8]:
from langsmith import Client
import os

ls_client = Client(api_key=os.environ["LANGSMITH_API_KEY"])


In [9]:
def next_agent_evaluator(run, example):

    next_agent_match = run.outputs["next_agent"] == example.outputs["next_agent"]
    final_answer_match = run.outputs["coordinator_final_answer"] == example.outputs["coordinator_final_answer"]

    return next_agent_match and final_answer_match

In [10]:
results = ls_client.evaluate(
    lambda x: coordinator_agent_node(State(messages=x["messages"])),
    data="coordinator-evaluation-dataset",
    evaluators=[next_agent_evaluator],
    experiment_prefix="coordinator-evaluation-dataset",
)

/Users/mcq/Development/AIML/SwirlAI/AI_bootcamp_summer_2025/01-ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'coordinator-evaluation-dataset-97df453a' at:
https://smith.langchain.com/o/8f1e49b5-0c40-40cc-93ed-220dd6b1054e/datasets/f4469453-7001-4a34-b117-1958b00eaad7/compare?selectedSessions=bd130ba6-edd8-4233-a898-7994377cdcf0




10it [00:17,  1.73s/it]


In [11]:
results.experiment_name

'coordinator-evaluation-dataset-97df453a'

In [12]:
results_resp = ls_client.read_project(
    project_name=results.experiment_name, include_stats=True
)

In [14]:
results_resp.feedback_stats["next_agent_evaluator"]["avg"]

0.7